# CNN Facial Recognition

## Introduciton
Hey guys, the Stats Whisper is back with another task at hand. This time we will be using a convulution neural network to resolve a doopleganger debate. 





## CNN Explianed

- images
- videos 
- mathematically explained


# Application

Importing appropriate libraries and image data

In [2]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import keras
from keras.models import Sequential,Input,Model
from keras import layers
from keras.layers import Dense, Flatten,Input,Dropout,Activation,Conv2D
from keras.callbacks import TensorBoard
import h5py
from time import time

batch_size = 12

train_datagen = ImageDataGenerator()

test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
        'data/train',  # this is the target directory
        target_size=(250, 250),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'data/test',
        target_size=(250, 250),
        batch_size=batch_size,
        class_mode='binary')

tensorboard = TensorBoard(log_dir="tensorBoard/".format(time()), write_images=True, write_graph=True)

Found 70 images belonging to 2 classes.
Found 70 images belonging to 2 classes.


In [7]:
from keras_vggface.vggface import VGGFace

base_model = VGGFace(include_top=False, input_shape=(125, 94, 3), pooling='max')

for layer in base_model.layers:
    layer.trainable = False

    
last_layer = base_model.get_layer('pool5').output
x = Flatten(name='flatten')(last_layer)
#x = Dense(hidden_dim, activation='relu')(x)
x=Dropout(0.5)(x)
x = Dense(hidden_dim, activation='relu')(x)
out = Dense(n_classes1, activation='softmax')(x)
custom_vgg_model = Model(base_model.input, out)

ImportError: cannot import name '_obtain_input_shape'